In [1]:
using MNIST, Plots               

In [23]:
f(x) = 1./(1+exp(-x))     # logistic function activation
df(y) = y.*(1-y)          # derivative of f composed with inverse of f

m = 60000  # number of examples in training set

n0 = 784       # widths of layers
n1 = 25
n2 = 10  

eta=0.1       # learning rate parameter
epsinit=0.01  # magnitude of initial conditions for synaptic weights

# two fully connected synaptic layers
W1 = epsinit*randn(n1,n0)
W2 = epsinit*randn(n2,n1)

# biases
b1 = epsinit*randn(n1,1)
b2 = epsinit*randn(n2,1)

tmax=600000
errsq = zeros(tmax)
errcl = zeros(tmax);

In [25]:
gr(                        # GR backend for Plots
    size = (600,600),    # you may need to change the numbers to fit your screen
    yflip = true,          # matrix convention that rows are numbered from the top
    color = :grays        # grayscale color map
)
for t = 1:tmax
    i = ceil(Int, m*rand())
    x0 = trainfeatures(i)/255
    y = zeros(n2,1)
    if trainlabel(i) == 0
        y[10] = 1
    else
        y[convert(Int,trainlabel(i))] = 1
    end
    # forward pass   
    x1 = f(W1*x0+b1)
    x2 = f(W2*x1+b2)
    # error computation
    errsq[t] = sum((y-x2).^2)
    delta2 = (y-x2).*df(x2)
    errcl[t] = Float64(rem(indmax(x2),10) != trainlabel(i))
#    println(rem(indmax(x2),10)," ",trainlabel(i)," ",errcl[t])
    # backward pass
    delta1 = (W2'*delta2).*df(x1)
    W2 += eta*delta2*x1'
    W1 += eta*delta1*x0'
    b2 += eta*delta2
    b1 += eta*delta1
    if rem(t,1000) == 0    # display every 1000 time steps
        avgerrsq = cumsum(errsq[1:t])./(1:t)
        avgerrcl = cumsum(errcl[1:t])./(1:t)
        IJulia.clear_output(true)
        l = @layout [a b c d; grid(5,5){0.75h}]
        plot(
            heatmap(
                reshape(x0,28,28),
                axis = nothing,
                legend = :none,
                title = "x0"
            ),
            bar(x1, color = :auto, yflip = false, ylabel="x1", legend=:none),
            bar(x2,color = :auto, yflip = false, ylabel="x2", legend=:none),
            plot([avgerrsq avgerrcl], color = :auto, 
                yflip = false, ylabel = "error", label = ["squared","class"],legend = :topright), 
            [heatmap(reshape(W1',28,28,n1)[:,:,i],axis=nothing) for i=1:n1]...,   # first layer weight vectors
            layout = l
        ) |> display
        sleep(0.01)
    end
end

LoadError: InterruptException: